# CNN (Convolutional Neural Nets)

- state of the art for image classification

### Architecture

1. `Convulational Layers` - image filters typically used with "relu" activation function

2. `Pooling Layer` - reduces the image dimensions by "pooling" or combining multiple pixel regions into a window (i.e. "maxpooling" just takes the maximum value from that window)

3. `Dense Layer` - perform classification on the features extracted by the convolutional layers and downsampled by the pooling layers. In a dense layer, every node in the layer is connected to every node in the preceding layer. "connective tissue"


### Reference

https://www.tensorflow.org/tutorials/estimators/cnn#intro_to_convolutional_neural_networks

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time

NAME = "Cats-v-Dog-cnn-62x2-{}".format(int(time.time()))

tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
import pickle

In [21]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [22]:
pickle_in = open("./data/PetImages/X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("./data/PetImages/y_pickle", "rb")
y = pickle.load(pickle_in)

In [34]:
X = X[:30000]
y = y[:30000]

In [35]:
X = X/255.0

In [36]:
model = Sequential()

model.add(Conv2D(64, (3,3), input_shape=X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten()) # need to flatten before Dense
# model.add(Dense(64))
# model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=3, validation_split=0.1, callbacks=[tensorboard])

Train on 10500 samples, validate on 4500 samples
Epoch 1/3
10500/10500 [==============================] - 34s 3ms/sample - loss: 0.6934 - acc: 0.5040 - val_loss: 0.6931 - val_acc: 0.4998
Epoch 2/3
10500/10500 [==============================] - 37s 3ms/sample - loss: 0.6932 - acc: 0.4952 - val_loss: 0.6932 - val_acc: 0.5002
Epoch 3/3
10500/10500 [==============================] - 37s 4ms/sample - loss: 0.6932 - acc: 0.5019 - val_loss: 0.6932 - val_acc: 0.5002


### Optimizing our CNN

In [37]:
dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conf-{}-nodes-{}-dense-{}".format(conv_layer,
                                                        layer_size,
                                                        dense_layer,
                                                        int(time.time()))
            print(NAME)
            model = Sequential()

            model.add(Conv2D(64, (3,3), input_shape=X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))
            
            for l in range(conv_layer-1):
                model.add(Conv2D(64, (3,3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))
            
            model.add(Flatten())
            for l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation("relu"))
            
            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            model.compile(loss='binary_crossentropy',
                         optimizer='adam',
                         metrics=['accuracy'])

            model.fit(X, y, batch_size=32, epochs=3, validation_split=0.3, callbacks=[tensorboard])

1-conf-32-nodes-0-dense-1555780897
Train on 10500 samples, validate on 4500 samples
Epoch 1/3
10500/10500 [==============================] - 16s 2ms/sample - loss: 0.6934 - acc: 0.4990 - val_loss: 0.6931 - val_acc: 0.5002
Epoch 2/3
10500/10500 [==============================] - 15s 1ms/sample - loss: 0.6932 - acc: 0.4989 - val_loss: 0.6932 - val_acc: 0.5002
Epoch 3/3
10500/10500 [==============================] - 15s 1ms/sample - loss: 0.6932 - acc: 0.4983 - val_loss: 0.6932 - val_acc: 0.5002
2-conf-32-nodes-0-dense-1555780946
Train on 10500 samples, validate on 4500 samples
Epoch 1/3
10500/10500 [==============================] - 34s 3ms/sample - loss: 0.6932 - acc: 0.5000 - val_loss: 0.6932 - val_acc: 0.5002
Epoch 2/3
10500/10500 [==============================] - 35s 3ms/sample - loss: 0.6932 - acc: 0.5036 - val_loss: 0.6932 - val_acc: 0.5002
Epoch 3/3
10500/10500 [==============================] - 37s 3ms/sample - loss: 0.6931 - acc: 0.5036 - val_loss: 0.6932 - val_acc: 0.5002
3-co

KeyboardInterrupt: 